In [2]:
# Import libraries
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

import pandas as pd
import numpy as np
import urllib.request
import zipfile
import os
from tqdm.notebook import tqdm
import pyarrow.parquet as pq
import zipfile
from pathlib import Path
import datetime
import dask.dataframe as dd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data_bicing_joined=dd.read_csv(f'/content/drive/MyDrive/CapstoneProject/data_bicing_joined_HX.csv',sep=';')

In [5]:
data_bicing_joined.head()

,station_id,name,year,month,day,hour,last_updated_formated,last_update_formated_H-1,last_update_formated_H-2,last_update_formated_H-3,...,is_installed,is_renting,is_returning,% Docks Availlable,% Docks Available H-4,% Docks Available H-3,% Docks Available H-2,% Docks Available H-1,status,ttl
0,290,"PL. DELS JARDINS D'ALFÀBIA, 1",2019,7,22,8,2019-07-22 08:59:55,2019-07-22 07:59:55,2019-07-22 06:59:55,2019-07-22 05:59:55,...,1,1,1,0.751131,0.352941,0.352941,0.352941,0.504902,IN_SERVICE,30
1,271,"VIA BARCINO, 69",2022,6,10,21,2022-06-10 21:59:45,2022-06-10 20:59:45,2022-06-10 19:59:45,2022-06-10 18:59:45,...,1,1,1,0.769231,0.753968,0.659341,0.645022,0.686508,IN_SERVICE,29
2,149,"C/ PUJADES, 57B",2022,6,8,20,2022-06-08 20:59:39,2022-06-08 19:59:39,2022-06-08 18:59:39,2022-06-08 17:59:39,...,1,1,1,0.735043,0.491582,0.675214,0.864198,0.801347,IN_SERVICE,30
3,342,"C/ ROC BORONAT, 134",2020,2,4,4,2020-02-04 04:55:16,2020-02-04 03:55:16,2020-02-04 02:55:16,2020-02-04 01:55:16,...,1,1,1,0.777778,0.898990,0.831909,0.777778,0.777778,IN_SERVICE,30
4,358,"C/ GOMBAU, 24",2021,5,28,8,2021-05-28 08:59:30,2021-05-28 07:59:30,2021-05-28 06:59:30,2021-05-28 05:59:30,...,1,1,1,0.943333,0.480000,0.480000,0.513333,0.766667,IN_SERVICE,30


In [6]:
weather=dd.read_csv(f'/content/drive/MyDrive/weather.csv', dtype={'mm_precip': 'float64'})

In [7]:
weather.head()

,timestamp,mm_precip,temperature
0,1546297200,0.0,9.1
1,1546299000,0.0,8.9
2,1546300800,0.0,9.0
3,1546302600,0.0,9.2
4,1546304400,0.0,8.6


In [8]:
import pandas as pd


In [9]:
# Convertir la columna 'timestamp' en objetos de fecha y hora 
weather['datetime'] = weather['timestamp'].map(lambda x: pd.to_datetime(x, unit='s'))

In [10]:
weather.head()

,timestamp,mm_precip,temperature,datetime
0,1546297200,0.0,9.1,2018-12-31 23:00:00
1,1546299000,0.0,8.9,2018-12-31 23:30:00
2,1546300800,0.0,9.0,2019-01-01 00:00:00
3,1546302600,0.0,9.2,2019-01-01 00:30:00
4,1546304400,0.0,8.6,2019-01-01 01:00:00


In [11]:
# Función personalizada para filtrar filas con minutos diferentes de 0 
def filter_hours(df):
    return df[df['datetime'].dt.minute == 0]

In [12]:
# Aplicar la función personalizada 
weather_hour = weather.map_partitions(filter_hours)

In [13]:
weather_hour.head()

,timestamp,mm_precip,temperature,datetime
0,1546297200,0.0,9.1,2018-12-31 23:00:00
2,1546300800,0.0,9.0,2019-01-01 00:00:00
4,1546304400,0.0,8.6,2019-01-01 01:00:00
6,1546308000,0.0,7.7,2019-01-01 02:00:00
8,1546311600,0.0,7.3,2019-01-01 03:00:00


In [14]:
# DataFrame de estaciones
data_bicing_joined['datetime'] = dd.to_datetime(data_bicing_joined['year'].astype(str) + '-' +
                                                data_bicing_joined['month'].astype(str) + '-' +
                                                data_bicing_joined['day'].astype(str) + ' ' +
                                                data_bicing_joined['hour'].astype(str) + ':00:00')

In [15]:
data_bicing_joined.head()

,station_id,name,year,month,day,hour,last_updated_formated,last_update_formated_H-1,last_update_formated_H-2,last_update_formated_H-3,...,is_renting,is_returning,% Docks Availlable,% Docks Available H-4,% Docks Available H-3,% Docks Available H-2,% Docks Available H-1,status,ttl,datetime
0,290,"PL. DELS JARDINS D'ALFÀBIA, 1",2019,7,22,8,2019-07-22 08:59:55,2019-07-22 07:59:55,2019-07-22 06:59:55,2019-07-22 05:59:55,...,1,1,0.751131,0.352941,0.352941,0.352941,0.504902,IN_SERVICE,30,2019-07-22 08:00:00
1,271,"VIA BARCINO, 69",2022,6,10,21,2022-06-10 21:59:45,2022-06-10 20:59:45,2022-06-10 19:59:45,2022-06-10 18:59:45,...,1,1,0.769231,0.753968,0.659341,0.645022,0.686508,IN_SERVICE,29,2022-06-10 21:00:00
2,149,"C/ PUJADES, 57B",2022,6,8,20,2022-06-08 20:59:39,2022-06-08 19:59:39,2022-06-08 18:59:39,2022-06-08 17:59:39,...,1,1,0.735043,0.491582,0.675214,0.864198,0.801347,IN_SERVICE,30,2022-06-08 20:00:00
3,342,"C/ ROC BORONAT, 134",2020,2,4,4,2020-02-04 04:55:16,2020-02-04 03:55:16,2020-02-04 02:55:16,2020-02-04 01:55:16,...,1,1,0.777778,0.898990,0.831909,0.777778,0.777778,IN_SERVICE,30,2020-02-04 04:00:00
4,358,"C/ GOMBAU, 24",2021,5,28,8,2021-05-28 08:59:30,2021-05-28 07:59:30,2021-05-28 06:59:30,2021-05-28 05:59:30,...,1,1,0.943333,0.480000,0.480000,0.513333,0.766667,IN_SERVICE,30,2021-05-28 08:00:00


In [16]:
def merge_temperature_rain(df_stations, df_weather, hours_difference):
    df_weather_shifted = df_weather.copy()
    df_weather_shifted['datetime'] = df_weather_shifted['datetime'] + pd.Timedelta(hours=hours_difference)
    df_stations = df_stations.merge(df_weather_shifted[['datetime', 'temperature','mm_precip']], on='datetime', how='left', suffixes=('', f'_H-{abs(hours_difference)}'))
    return df_stations


In [17]:
data_bicing_joined_climate = merge_temperature_rain(data_bicing_joined, weather_hour, 0)


In [18]:
for i in range(1, 5):
    data_bicing_joined_climate = merge_temperature_rain(data_bicing_joined_climate, weather_hour, -i)

In [19]:
data_bicing_joined_climate.head()

,station_id,name,year,month,day,hour,last_updated_formated,last_update_formated_H-1,last_update_formated_H-2,last_update_formated_H-3,...,temperature,mm_precip,temperature_H-1,mm_precip_H-1,temperature_H-2,mm_precip_H-2,temperature_H-3,mm_precip_H-3,temperature_H-4,mm_precip_H-4
0,290,"PL. DELS JARDINS D'ALFÀBIA, 1",2019,7,22,8,2019-07-22 08:59:55,2019-07-22 07:59:55,2019-07-22 06:59:55,2019-07-22 05:59:55,...,30.8,0.0,31.5,0.0,32.2,0.0,32.5,0.0,32.3,0.0
1,271,"VIA BARCINO, 69",2022,6,10,21,2022-06-10 21:59:45,2022-06-10 20:59:45,2022-06-10 19:59:45,2022-06-10 18:59:45,...,21.1,0.0,21.2,0.0,20.7,0.0,19.9,0.0,20.0,0.0
2,149,"C/ PUJADES, 57B",2022,6,8,20,2022-06-08 20:59:39,2022-06-08 19:59:39,2022-06-08 18:59:39,2022-06-08 17:59:39,...,20.8,0.0,20.6,0.0,19.9,0.0,20.0,0.0,19.9,0.0
3,342,"C/ ROC BORONAT, 134",2020,2,4,4,2020-02-04 04:55:16,2020-02-04 03:55:16,2020-02-04 02:55:16,2020-02-04 01:55:16,...,16.8,0.0,15.8,0.0,14.6,0.0,14.3,0.0,15.2,0.0
4,358,"C/ GOMBAU, 24",2021,5,28,8,2021-05-28 08:59:30,2021-05-28 07:59:30,2021-05-28 06:59:30,2021-05-28 05:59:30,...,23.5,0.0,23.9,0.0,24.4,0.0,24.6,0.0,24.5,0.0


In [20]:
#Funcion para saber si es fin de semana o no
def is_weekend(day_of_week):
    return 1 if day_of_week >= 5 else 0

data_bicing_joined_climate['day_of_week'] = data_bicing_joined_climate['datetime'].dt.dayofweek
data_bicing_joined_climate['is_weekend'] = data_bicing_joined_climate['day_of_week'].map(is_weekend, meta=('is_weekend', 'int64'))
data_bicing_joined_climate = data_bicing_joined_climate.drop('day_of_week', axis=1)

In [24]:
data_bicing_joined_climate.head()

,station_id,name,year,month,day,hour,last_updated_formated,last_update_formated_H-1,last_update_formated_H-2,last_update_formated_H-3,...,mm_precip,temperature_H-1,mm_precip_H-1,temperature_H-2,mm_precip_H-2,temperature_H-3,mm_precip_H-3,temperature_H-4,mm_precip_H-4,is_weekend
0,290,"PL. DELS JARDINS D'ALFÀBIA, 1",2019,7,22,8,2019-07-22 08:59:55,2019-07-22 07:59:55,2019-07-22 06:59:55,2019-07-22 05:59:55,...,0.0,31.5,0.0,32.2,0.0,32.5,0.0,32.3,0.0,0
1,271,"VIA BARCINO, 69",2022,6,10,21,2022-06-10 21:59:45,2022-06-10 20:59:45,2022-06-10 19:59:45,2022-06-10 18:59:45,...,0.0,21.2,0.0,20.7,0.0,19.9,0.0,20.0,0.0,0
2,149,"C/ PUJADES, 57B",2022,6,8,20,2022-06-08 20:59:39,2022-06-08 19:59:39,2022-06-08 18:59:39,2022-06-08 17:59:39,...,0.0,20.6,0.0,19.9,0.0,20.0,0.0,19.9,0.0,0
3,342,"C/ ROC BORONAT, 134",2020,2,4,4,2020-02-04 04:55:16,2020-02-04 03:55:16,2020-02-04 02:55:16,2020-02-04 01:55:16,...,0.0,15.8,0.0,14.6,0.0,14.3,0.0,15.2,0.0,0
4,358,"C/ GOMBAU, 24",2021,5,28,8,2021-05-28 08:59:30,2021-05-28 07:59:30,2021-05-28 06:59:30,2021-05-28 05:59:30,...,0.0,23.9,0.0,24.4,0.0,24.6,0.0,24.5,0.0,0


In [ ]:
output_path = '/content/drive/MyDrive/data_bicing_joined_climate.csv'
data_bicing_joined_climate.to_csv(output_path, index=False, compute=True, single_file=True)